# Pickling cuML Models for Persistence

This notebook demonstrates simple pickling of both single-GPU and multi-GPU cuML models for persistence

In [1]:
import warnings
warnings.filterwarnings("ignore", category=FutureWarning)

## Single GPU Model Pickling

All single-GPU estimators are pickleable. The following example demonstrates the creation of a synthetic dataset, training, and pickling of the resulting model for storage. Trained single-GPU models can also be used to distribute the inference on a Dask cluster, which the `Distributed Model Pickling` section below demonstrates.

In [2]:
from cuml.datasets import make_blobs

X, y = make_blobs(n_samples=50,
                  n_features=10,
                  centers=5,
                  cluster_std=0.4,
                  random_state=0)

In [3]:
from cuml.cluster import KMeans

model = KMeans(n_clusters=5)

model.fit(X)

KMeans()

In [4]:
import pickle

pickle.dump(model, open("kmeans_model.pkl", "wb"))

In [5]:
model = pickle.load(open("kmeans_model.pkl", "rb"))

In [6]:
model.cluster_centers_

array([[-5.7684636 ,  2.3276033 , -3.7457774 , -1.8541754 , -5.1695833 ,
         7.6670885 ,  2.7118318 ,  8.495609  ,  1.7038484 ,  1.1884269 ],
       [ 4.647688  ,  8.37788   , -9.070581  ,  9.459332  ,  8.450423  ,
        -1.0210547 ,  3.3920872 , -7.862985  , -0.75276613,  0.4838412 ],
       [-2.9414442 ,  4.6401706 , -4.5027533 ,  2.2855108 ,  1.644645  ,
        -2.4937892 , -5.2241607 , -1.5499196 , -8.063638  ,  2.8169358 ],
       [-4.2710767 ,  5.5611653 , -5.6640916 , -1.8229512 , -9.2925    ,
         0.730283  ,  4.4586773 , -2.8876224 , -5.1257744 ,  9.694357  ],
       [ 5.583742  , -4.1515303 ,  4.369667  , -3.0020504 ,  3.638897  ,
        -4.3419113 , -3.3187115 ,  6.503671  , -6.865036  , -1.0266498 ]],
      dtype=float32)

## Distributed Model Pickling

The distributed estimator wrappers inside of the `cuml.dask` are not intended to be pickled directly. The Dask cuML estimators provide a function `get_combined_model()`, which returns the trained single-GPU model for pickling. The combined model can be used for inference on a single-GPU, and the `ParallelPostFit` wrapper from the [Dask-ML](https://ml.dask.org/meta-estimators.html) library can be used to perform distributed inference on a Dask cluster.

In [7]:
from dask.distributed import Client
from dask_cuda import LocalCUDACluster

cluster = LocalCUDACluster()
client = Client(cluster)
client

Client Scheduler: tcp://127.0.0.1:42631 Dashboard: http://127.0.0.1:8787/status,Cluster Workers: 1 Cores: 1 Memory: 270.37 GB


In [8]:
from cuml.dask.datasets import make_blobs

n_workers = len(client.scheduler_info()["workers"].keys())

X, y = make_blobs(n_samples=5000, 
                  n_features=30,
                  centers=5, 
                  cluster_std=0.4, 
                  random_state=0,
                  n_parts=n_workers*5)

X = X.persist()
y = y.persist()

In [9]:
from cuml.dask.cluster import KMeans

dist_model = KMeans(n_clusters=5)

In [10]:
dist_model.fit(X)

In [11]:
import pickle

single_gpu_model = dist_model.get_combined_model()
pickle.dump(single_gpu_model, open("kmeans_model.pkl", "wb"))

In [12]:
single_gpu_model = pickle.load(open("kmeans_model.pkl", "rb"))

In [13]:
single_gpu_model.cluster_centers_

array([[ 4.8098736 ,  8.42267   , -9.239021  ,  9.379146  ,  8.499882  ,
        -1.0592816 ,  3.3437858 , -7.802612  , -0.5946332 ,  0.264476  ,
         5.5073957 , -4.10698   ,  4.289078  , -2.8172052 ,  3.6150155 ,
        -4.1613    , -3.6209636 ,  6.2185307 , -6.946048  , -1.0828305 ,
        -5.826771  ,  2.2258766 , -3.8601208 , -1.6974077 , -5.3134184 ,
         7.579579  ,  2.9187474 ,  8.540424  ,  1.5523208 ,  1.0841806 ],
       [-2.8941855 ,  4.4741898 , -4.447567  ,  2.3820984 ,  1.7478832 ,
        -2.5046253 , -5.208329  , -1.6937686 , -8.134755  ,  2.6468296 ,
        -4.3163633 ,  5.5655394 , -5.732199  , -1.738495  , -9.344657  ,
         0.70846575,  4.435839  , -2.9009004 , -4.9486394 ,  9.695306  ,
         8.36652   , -6.247453  , -6.3494735 ,  1.9546976 ,  4.157616  ,
        -9.167903  ,  4.6070676 ,  8.788586  ,  6.8644223 ,  2.231988  ],
       [-4.6657143 , -9.558954  ,  6.6572313 ,  4.440131  ,  2.1730309 ,
         2.5904033 ,  0.58000994,  6.2550364 , -8